In [8]:
import pandas as pd
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np

MOVING_WIN_SIZE = 5
DS_SPLIT = 0.8

# Read CSV data
typeg_data = pd.read_csv("./data/2330.TW.csv")

# Define additional features (replace with your desired features)
typeg_data["Open_Diff"] = typeg_data["Open"] - typeg_data["Close"].shift(1)
typeg_data["Volume_Change"] = typeg_data["Volume"].pct_change()

# Filter target variable and reset index
test_data = typeg_data[["Close"]].copy()
test_data.reset_index(drop=True, inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(typeg_data[["Close", "Open_Diff", "Volume_Change"]])

all_x, all_y = [], []
for i in range(len(scaled_data) - MOVING_WIN_SIZE):
    x = scaled_data[i : i + MOVING_WIN_SIZE, :]  # Include additional features
    y = scaled_data[i + MOVING_WIN_SIZE, 0]  # Target: next day's closing price
    all_x.append(x)
    all_y.append(y)

all_x, all_y = np.array(all_x), np.array(all_y)
train_size = round(all_x.shape[0] * DS_SPLIT)
train_x, train_y = all_x[:train_size], all_y[:train_size]
test_x, test_y = all_x[train_size:], all_y[train_size:]

# Model definition with dropout for regularization
model = Sequential()
model.add(LSTM(units=128, return_sequences=False, dropout=0.2, input_shape=(train_x.shape[1], train_x.shape[2])))  # Specify 3D input shape
model.add(LSTM(units=50, return_sequences=False, dropout=0.2))
model.add(Dense(units=25, activation="relu"))  # Add activation for hidden layer
model.add(Dense(units=1))

model.compile(optimizer="adam", loss="mean_squared_error")

callback = EarlyStopping(monitor="val_loss", patience=100, restore_best_weights=True)
model.fit(
    train_x,
    train_y,
    validation_split=0.2,
    callbacks=[callback],
    epochs=1000,
)
preds = scaler.inverse_transform(np.reshape(model.predict(test_x), (test_x.shape[0], 1, -1)))
# preds = scaler.inverse_transform(model.predict(test_x))

# Evaluation with additional metrics (replace with your desired metrics)
test_data["Predict"] = preds
test_data["Actual"] = test_data["Close"]
test_data["Correct"] = test_data["Predict"].round() == test_data["Actual"]
accuracy = accuracy_score(test_data["Correct"]) * 100

print(f"Accuracy: {accuracy:.2f} %")

# Additional analysis (optional):
# - Plot actual vs predicted values
# - Calculate other metrics (e.g., MAPE)


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Input 0 of layer "lstm_13" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 128)